# Lymphocyte differentiation
## A tutorial on Agent based models in python
### Python "club" - October 2016
1st version: 20/10/2016
Last version: 21/10/2016

In [1]:
class Cell(object):
    ''' The mother of all the cells'''
    def __init__(self):
        ''' The constructor'''
        self.type = 'generic'
        self.place = 'wherever'
        self.state = 'undefined'
    def __str__(self):
        ''' When you call using print'''
        return "Type: %s Place: %s State: %s"%(self.type,self.place,self.state)
    def death(self):
        ''' Kill a cell'''
        print "I'ts the end of the world as we know it!"
        self.state='dead'

In [14]:
c = Cell()
print c
print c.place # The dot allows us to access to the class attributes and methods
print c.state
c.death()
print c.state

Type: generic Place: wherever State: undefined
wherever
undefined
I'ts the end of the world as we know it!
dead


### Let's play with inheritance

In [3]:
class lymphoid(Cell):
    def __init__(self):
        self.type = 'lymphoid progenitor'
        self.place = 'bone marrow'
        self.state = 'immature'

In [15]:
l = lymphoid()
print l
print l.state
l.death() # lymphoid has inherited the method "death" from its ancestor
print l.state

Type: lymphoid progenitor Place: bone marrow State: immature
immature
I'ts the end of the world as we know it!
dead


### Create a bunch of lymphoid cells

In [6]:
lymphoid_list = [lymphoid() for i in range(10)]

This command is called 'list comprehension
The syntax is simple: call to "lymphoid()" for every item 'i' in the list range(10)=0,1,...9

### Lets modify our class to do more interesting stuff. But first, let's play with random numbers

In [10]:
from random import random,randint
print random()
print randint(1,8)
from math import log
log(1.)

0.458430879605
2


0.0

### Let's extend the lymphoid class a little bit

In [16]:
class lymphoid(Cell):
    def __init__(self):
        self.type = 'lymphoid progenitor'
        self.wheretogo = 0.6 # Probability of staying at the bone marrow or go to the thymus
        self.place = 'bone marrow'
        self.state = 'immature'
    def move(self):
        if random()>=0.6:
            self.place = 'thymus'
            

### Give these cells some friends...

In [18]:
class APC(Cell):
    ''' Antigen presenting cell'''
    def __init__(self,place='thymus'):
        self.type = 'Antigen presenting cell'
        self.place = place
        self.pMHC = random()
        self.HLA = randint(0,1)
        self.state = 'undefined'
    

In [19]:
l = lymphoid()
print l
l.move()
print l

Type: lymphoid progenitor Place: bone marrow State: immature
Type: lymphoid progenitor Place: thymus State: immature


#### Sometimes, the best way to deal with agents is to collect them into a new class.

In [27]:
class CellPopulation(object):
    def __init__(self):
        self.thelist = [] # To store the elements of the population
        self.len = 0 # The length 
    
    def add(self,newcell):
        self.thelist.append(newcell)
        self.len += 1 # we could do simply len(self.thelist), but that takes time
    
    def __str__(self):
        ''' Define the output when printing '''
        out = ""
        for i in range(self.len):
            out += self.thelist[i].__str__()+"\n" # Concatenate the outputs of every cell.
        return out
    
    def killcell(self,i):
        self.thelist[i].state ='dead'
        self.updatecell(i)
    
    def updatecell(self,i):
        if self.thelist[i].state=='dead':
            self.thelist[i]=self.thelist[-1] # A good-old trick, replace the current by the last...
            self.thelist.pop(-1) # ... and pop the last element (this is the fastest way to kill)
            self.len -= 1 # Update the count

In [28]:
cp = CellPopulation()
[cp.add(lymphoid()) for i in range(5)];

In [29]:
print cp
cp.killcell(4)
print cp

Type: lymphoid progenitor Place: bone marrow State: immature
Type: lymphoid progenitor Place: bone marrow State: immature
Type: lymphoid progenitor Place: bone marrow State: immature
Type: lymphoid progenitor Place: bone marrow State: immature
Type: lymphoid progenitor Place: bone marrow State: immature

Type: lymphoid progenitor Place: bone marrow State: immature
Type: lymphoid progenitor Place: bone marrow State: immature
Type: lymphoid progenitor Place: bone marrow State: immature
Type: lymphoid progenitor Place: bone marrow State: immature



### Create the T lymphocytes

In [30]:
class tlymph(lymphoid):
    def __init__(self):
        self.type = 'T lymphocyte'
        self.place = 'thymus'
        self.state = 'beta_arrangement'
        self.TCR = random()
        
    def checkpoint(self,APC):
        #print 'pre-state: ',self.state
        if self.state =='beta_arrangement':
            self.state = 'alpha_arrangement'
            if APC.pMHC > self.TCR:
                self.state = 'dead'
                
        if self.state=='alpha_arrangement':
            if APC.HLA == 1:
                self.state = 'Killer T-cell'
            else:
                self.state = 'Helper T-cell'
        #print 'post-state: ',self.state

### Let's play with the cells

In [36]:
t = tlymph()
a = APC()
b = APC()
print t
print a
print b

Type: T lymphocyte Place: thymus State: beta_arrangement
Type: Antigen presenting cell Place: thymus State: undefined
Type: Antigen presenting cell Place: thymus State: undefined


In [39]:
print t
t.checkpoint(a)
print t
t.checkpoint(b)
print t

Type: T lymphocyte Place: thymus State: dead
Type: T lymphocyte Place: thymus State: dead
Type: T lymphocyte Place: thymus State: dead


### Create some populations

In [40]:
tpop = CellPopulation()
[tpop.add(tlymph()) for i in range(10)];
apop = CellPopulation()
[apop.add(APC()) for i in range(10)];

## The scheduler: Biology is about (time) evolution

In [41]:
class Scheduler(object):
    def __init__(self):
        self.time = 0
        self.rate = 100 # total probability of encounter per day
    def update(self):
        ti = randint(0,tpop.len-1) # Pick one tlymph
        ai = randint(0,apop.len-1) # Pick one APC
        tpop.thelist[ti].checkpoint(apop.thelist[ai]) # Let the tlymph meet an APC
        tpop.updatecell(ti) # Update its state
        self.time += -log(random())/self.rate # A quick and dirty Gillespie algorithm

In [43]:
simulation = Scheduler()
for item in tpop.thelist: # Print the TCR "spectrum"
    print item.TCR
print "Simulation time=",simulation.time

0.630563930575
0.650156057656
0.506523579818
0.689796663423
0.222744740428
0.989092589023
0.711242798912
0.184432689801
0.792536279482
0.340355231724
t= 0


In [44]:
for steps in range(100): # Run 100 times
    simulation.update()
print "Simulation time=",simulation.time

Simulation time= 1.06021001865


In [45]:
print tpop

Type: T lymphocyte Place: thymus State: Killer T-cell
Type: T lymphocyte Place: thymus State: Killer T-cell
Type: T lymphocyte Place: thymus State: Killer T-cell
Type: T lymphocyte Place: thymus State: Helper T-cell
Type: T lymphocyte Place: thymus State: Killer T-cell
Type: T lymphocyte Place: thymus State: Helper T-cell



In [47]:
for item in tpop.thelist:
    print item.TCR

0.711242798912
0.650156057656
0.792536279482
0.689796663423
0.340355231724
0.989092589023


#### Some of them died, and we have the spectrum of the survivors